In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cambio = pd.read_excel('../datasets/tc_turistas.xlsx',
                       sheet_name=0)

turista = pd.read_excel('../datasets/tc_turistas.xlsx',
                        sheet_name=2)

# Limpieza de datos

## Cambio

In [ ]:
cambio.rename(
    columns={
        'FECHA': 'fecha',
        'DOLAR Oficial': 'dolar_oficial',
        'DOLAR Blue': 'dolar_blue'
    },
    inplace=True
)

Para completar los múltiples datos faltantes de los precios decidimos usar el último valor observado.

Otra forma posible hubise sido usar alguna medida resumen para los valores faltantes, pero en este caso eso haría que fluctúe mucho el incremento del dólar, cuando en realidad tiene una tendencia creciente.

In [ ]:
cambio.dolar_oficial.fillna(method='ffill', inplace=True)
cambio.dolar_blue.fillna(method='ffill', inplace=True)

Agrupamos por mes y calculamos el promedio de los valores.

In [ ]:
cambio_mensual = cambio.set_index('fecha').resample('M').mean()

In [ ]:
cambio_mensual.index = cambio_mensual.index.strftime('%Y-%m')

In [ ]:
cambio_mensual.iloc[:, 0:2] = \
    cambio_mensual.iloc[:, 0:2].round(2)

## Turista

In [ ]:
turista.rename(
    columns={col: col.lower() for col in turista.columns},
    inplace=True
)

Casteamos las columnas de los turistas llegados de cada país a `int`.

In [ ]:
turista[turista.columns[3:]] =\
    turista[turista.columns[3:]].astype('int64')

Viendo los máximos de cada campo vemos que hay un valor atípico en uno de ellos.

In [ ]:
turista.max()

El valor atípico lo tiene la entrada de año 2018 y mes abril.

In [ ]:
turista[turista.chile == 571838802363218]

Decidimos tomar el promedio del año previo y sucesor de 2018 para reemplazar el valor atípico.

In [ ]:
turista.loc[turista.chile == 571838802363218, 'chile'] = \
    turista[(turista.anio.isin([2017, 2019])) & (turista.mes == 'abr')]\
        .chile.mean()\
        .astype('int64')

Pasmaos la columna `date` a formato `datetime`.

In [ ]:
turista.date = \
    pd.to_datetime(turista.date, unit='d', origin='1899-12-30')

In [ ]:
turista['fecha_mensual'] = \
    turista.date.dt.strftime('%Y-%m')

In [ ]:
turista

In [ ]:
turista_cambio = \
    pd.merge(cambio_mensual, turista, 
            left_index=True, right_on='fecha_mensual', 
            how='left')

In [ ]:
turista_cambio = \
    turista_cambio\
        .drop(columns=['anio', 'mes', 'date'])\
        .reindex(columns=['fecha_mensual', 'dolar_oficial',
                        'dolar_blue', 'bolivia', 'brasil',
                        'chile', 'paraguay', 'uruguay'])

In [79]:
turista_cambio.set_index('fecha_mensual')

,dolar_oficial,dolar_blue,bolivia,brasil,chile,paraguay,uruguay
fecha_mensual,,,,,,,
2017-06,16.29,16.50,91354,44950,51901,44105,22149
2017-07,17.38,17.17,139655,67292,90150,74440,25872
2017-08,17.63,18.22,101980,53708,67773,60127,26348
2017-09,17.43,17.95,113370,79188,66416,70244,20974
2017-10,17.66,17.96,108988,86278,74250,65150,27705
2017-11,17.69,18.05,100106,73925,67212,58810,41117
2017-12,17.95,18.31,113630,106258,162180,54931,53165
2018-01,19.27,19.65,156813,124143,98232,47638,72066
2018-02,20.09,20.16,96556,170947,80980,49857,56729


### *Preguntar que es la columna date de turista, que hacer con el valor atípico de 2018-abr-chile y por que en el intervalo de cuarenta date no es 0*

# Exploración de datos